flowchart TB
  %% Lanes
  subgraph L1[Client (Chat UI)]
    U[사용자 입력<br/>예: '시원한 바다향 10만원대 남자 향수']
    UI[챗 UI<br/>세션/컨텍스트 유지]
    U --> UI
  end

  subgraph L2[Superagent Assistant (Orchestration)]
    P[LLM 파싱 (intent/slots)<br/>→ main_accord, price, brand, others, confidence]
    R[라우팅 정책<br/>sim < 임계치 → 키워드 확장]
    S1[1차 검색<br/>임베딩 유사도 + BM25]
    KW[키워드 확장 (벡터DB)<br/>accord_tag 상위 2~3]
    S2[재검색 (확장 쿼리)<br/>태그 부스트]
    RK[필터 & 랭킹<br/>가격·브랜드·개인화 반영 → Top3]
    PRICE[가격 조회<br/>Price Service (실시간 API+캐시)]
    RESP[응답 생성 (LLM 템플릿)<br/>Top3 설명 + 상세 링크 제공]

    UI --> P --> R
    R -->|유사도 양호| S1
    R -->|유사도 낮음| KW --> S2
    S1 --> RK
    S2 --> RK
    RK --> PRICE --> RESP
  end

  subgraph L3[Backend Services & Data]
    DB[(Perfume DB<br/>설명/링크/임베딩)]
    VEC[(Vector DB<br/>키워드 사전 (accords))]
    CACHE[(Price Cache DB<br/>TTL 저장)]
    PROV[(가격 제공자 API<br/>Amazon, Naver 등)]

    DB -. 조회 .- S1
    VEC -. 조회 .- KW
    PRICE -. 캐시조회/저장 .- CACHE
    PRICE -. 외부호출 .- PROV
  end

  %% Logging & Feedback (횡단)
  LOG[로깅<br/>쿼리/후보/스코어/응답 기록]
  FB[피드백 저장<br/>좋아요/싫어요/즐겨찾기 → 선호 벡터 업데이트]

  P -. 기록 .- LOG
  S1 -. 기록 .- LOG
  S2 -. 기록 .- LOG
  RK -. 기록 .- LOG
  RESP -. 기록 .- LOG

  RESP --> UI
  UI --> FB --> RK
